In [1]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
os.getcwd()

'/Users/lalitayang/Documents/K2/Unit 4/Project'

In [3]:
def load_credit_data(filename, path='/Users/lalitayang/Documents/K2/Unit 4/Project/data/'):
    """ import data provided by Kaggle"""
    csv_path = os.path.join(path, filename)
    return pd.read_csv(csv_path)

In [43]:
train = load_credit_data('application_train.csv')
test = load_credit_data('application_test.csv')

In [42]:
bur_prep = pd.read_csv('bureau_prepped.csv')

In [44]:
# drop outliers/bad data: 19 children and XNA as Gender
train = train.loc[(train['CNT_CHILDREN'] != 19) & (train['CODE_GENDER'] != 'XNA')]

In [45]:
train = pd.merge(train, bur_prep, left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='left')

In [58]:
train.shape

(307505, 160)

## Create train and test sets
Since the target is highly skewed (8% of data had difficulty paying back loans), I'll be using StratifiedShuffleSplit to ensure the train and test sets have a mix of targets

In [6]:
from sklearn.model_selection import StratifiedShuffleSplit

In [46]:
X = train.drop('TARGET', axis=1)
y = train['TARGET']

In [47]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X, y):
    train_set = train.iloc[train_index]
    test_set = train.iloc[test_index]
    
    
print('train: ', train_set.shape)
print('test: ', test_set.shape)

train:  (246004, 160)
test:  (61501, 160)


In [48]:
X_train = train_set.drop('TARGET', axis=1)

In [49]:
y_train = train_set['TARGET']

## List of features to drop

After reading the column descriptions, there were many features that were related to normalized descriptions of the property (min/max/mean/count of windows/doors/etc). As such, I determined that it was appropriate to remove these features to reduce the feature space.

In addition, I will be dropping features that have greater than 50% null values

In [11]:
def calc_nulls(df, percentage=0.0):
    """ returns: dataframe of features with null values and shape
        null count
        percentage: specify % to get features that containing % greater
    """
    nulls = df.isnull().sum().to_frame().reset_index()
    nulls.columns = ['columns', 'null_count']
    nulls['percent'] = nulls['null_count'].apply(lambda x: x/df.shape[0])
    nulls = nulls[nulls['null_count'] >0] # add percentage of missing. if~50% missing then maybe drop?
    nulls = nulls[nulls['percent'] > percentage]
    print(nulls.shape)
    return nulls

In [12]:
nulls = calc_nulls(X_train, 0.5)
drop_null_vars = list(nulls[nulls['percent'] > 0.5]['columns']) #drop features with >50% null
drop_null_vars.append('EMERGENCYSTATE_MODE') #less than 50% missing but uninterpretable/difficulty to impute
len(drop_null_vars)

(41, 3)


42

## Pipeline - drop null features

Build custom transformer for the pipeline

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, Imputer
import category_encoders as ce
from sklearn.pipeline import FeatureUnion, Pipeline

In [14]:
class NullFeatDrop(BaseEstimator, TransformerMixin):
    def __init__(self, null_cols=drop_null_vars): 
        self.null_cols = null_cols
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        dat = X.copy()
        cols = list(set(dat.columns) - set(self.null_cols))
        return dat.loc[:, cols]     

In [15]:
nfd = NullFeatDrop()
X_train2 = nfd.fit_transform(X_train)
X_train2.shape

(246004, 79)

## Pipeline - Categorical features

Build custom transformer for categorical features. For each selected categorical feature, One Hot Encode the categories and fill in nulls with blank

In [16]:
text_cols = X_train2.select_dtypes(exclude=[int, float]).columns
text_feats = [e for e in text_cols if e not in drop_null_vars]
text_feats

['NAME_HOUSING_TYPE',
 'ORGANIZATION_TYPE',
 'FLAG_OWN_REALTY',
 'WEEKDAY_APPR_PROCESS_START',
 'NAME_TYPE_SUITE',
 'CODE_GENDER',
 'NAME_CONTRACT_TYPE',
 'OCCUPATION_TYPE',
 'NAME_EDUCATION_TYPE',
 'FLAG_OWN_CAR',
 'NAME_INCOME_TYPE',
 'NAME_FAMILY_STATUS']

In [17]:
# Definition of the CategoricalEncoder class, copied from PR #9151.

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [18]:
class OheFeatures(BaseEstimator, TransformerMixin):
    """ Custom transformer for categorical variables
        Fill in nulls with the string 'Blank' 
        Encode the categories
    """
    def __init__(self, text_feats=text_feats):
        self.text_feats = text_feats
    def fit(self, X, y=None):
        return self  
    def transform(self, X, y=None):
        df = X.copy()
        df.fillna('Blank', inplace=True) #fill in blanks
        cat_encoder = CategoricalEncoder()
        df_reshaped = df.loc[:,text_feats].values
        df_1hot = cat_encoder.fit_transform(df_reshaped)
        return df_1hot

In [19]:
# test 
ohef = OheFeatures()
cf2 = ohef.fit_transform(X_train2) 
cf2.shape

(246004, 125)

## Pipeline - Numerical features

Build custom transformer for selected numerical features

In [20]:
imp_mean_cols = ['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_LAST_PHONE_CHANGE']

In [21]:
class TypeSelector(BaseEstimator, TransformerMixin):
    """ Select feature type """
    def __init__(self, dtype):
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])
    
class NumImputer(BaseEstimator, TransformerMixin):
    """ Custom Transformer to impute with mean or median, based on a list of indicated features""" 
    def __init__(self, imp_mean_cols=imp_mean_cols): 
        self.imp_mean_cols = imp_mean_cols
    def fit(self, X, y=None):
        return self 
    def transform(self, X, y=None):
        df = X.copy()
        imp_mean = Imputer(strategy='mean')
        imp_mean = imp_mean.fit(df[imp_mean_cols])
        df[imp_mean_cols] = imp_mean.transform(df[imp_mean_cols])
        imp_med = Imputer(strategy='median')
        imp_med = imp_med.fit(df)
        df = imp_med.transform(df)
        return df

In [22]:
# test
ni = Pipeline([
        ('numericals', Pipeline([
            ('selector', TypeSelector(np.number)),
            ('imp', NumImputer()),
            ('scaler', StandardScaler())
        ]))])
ni3 = ni.fit_transform(X_train2)
ni3.shape

(246004, 67)

# Final pipeline

Put all the custom transformers together

In [23]:
transformer = Pipeline([
    ('nullfeats', NullFeatDrop(drop_null_vars)),
    
    ('features', FeatureUnion(n_jobs=1, transformer_list=[        
        ('numericals', Pipeline([
            ('selector', TypeSelector(np.number)),
            ('imputer', NumImputer()),
            ('scaler', StandardScaler()),
        ])),  # numericals close
        
        ('categoricals', OheFeatures())  # categoricals close
    ]))  # features close
    
    #,('clf', RandomForestClassifier())
])  # pipeline close

In [50]:
trans = transformer.fit_transform(X_train)

In [59]:
trans.shape

(246004, 230)

## Try different models

In [25]:
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [51]:
X_test = test_set.drop('TARGET', axis=1)
y_test = test_set['TARGET']

In [52]:
X_test = transformer.fit_transform(X_test)

In [28]:
from sklearn.linear_model import LogisticRegression

In [53]:
lr = LogisticRegression()

In [54]:
lr.fit(trans,y_train)
lr_predict = lr.predict_proba(X_test)[:,1]

In [55]:
roc_auc_score(y_test, lr_predict)

0.74814804879379926

In [56]:
rfc = RandomForestClassifier()
rfc.fit(trans,y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [33]:
#without bureau data
rfc_predict = rfc.predict_proba(X_test)[:,1]
roc_auc_score(y_test,rfc_predict)

0.62062622879756424

In [57]:
#with bureau data
rfc_predict = rfc.predict_proba(X_test)[:,1]
roc_auc_score(y_test,rfc_predict)

0.62051007149095605

In [34]:
from sklearn.model_selection import cross_val_score

In [40]:
#scores = cross_val_score(rfc, trans, y_train, scoring='roc_auc', cv=3)

In [ ]:
#scores.mean()

In [36]:
from sklearn.ensemble import VotingClassifier

In [37]:
voting_clf = VotingClassifier(
    estimators=[('lr', lr), ('rf', rfc)],
    voting='soft')
voting_clf.fit(trans, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

In [38]:
voting_predict = voting_clf.predict_proba(X_test)[:,1]

In [39]:
roc_auc_score(y_test, voting_predict)

0.6973251667858682

### Conclusion

Looks like the best score I got without tuning the hyperparameters was 0.748

With more time, I would incorporate more data and do more feature engineering/selection, try more models, and tune the parameters